<a href="https://colab.research.google.com/github/TharinsaMudalige/Neuron-Brain_Tumor_Detection_Classification_with_XAI/blob/Detection-Classficiation-CNN/CNN_to_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Libraries & Set Paths

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Define dataset and model save paths (adjust as needed)
SPLIT_DATASET_PATH = '/content/drive/MyDrive/DSGP/Splitted_Dataset'
MODEL_SAVE_PATH = '/content/drive/MyDrive/DSGP/multiclass_model.h5'

# Training parameters
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

Create Data Generators

In [ ]:
# Create ImageDataGenerators with rescaling
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

# Flow data from the directory structure
train_data = train_datagen.flow_from_directory(
    directory=os.path.join(SPLIT_DATASET_PATH, 'train'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    directory=os.path.join(SPLIT_DATASET_PATH, 'val'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    directory=os.path.join(SPLIT_DATASET_PATH, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    shuffle=False  # keep order for evaluation
)

Found 25600 images belonging to 16 classes.
Found 3200 images belonging to 16 classes.
Found 3200 images belonging to 16 classes.


Build & Compile the CNN Model

In [ ]:
# Get the number of classes from the training data
num_classes = train_data.num_classes

# Build the model
model = models.Sequential([
    layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 16)                  │           2,064 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,171,024 (42.61 MB)

 Trainable params: 11,171,024 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Train & Save the Model

In [ ]:
EPOCHS = 10  # Adjust as needed

history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=val_data
)

# Save the trained model to Google Drive
model.save(MODEL_SAVE_PATH)
print("Model saved successfully at:", MODEL_SAVE_PATH)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 9220s 12s/step - accuracy: 0.7256 - loss: 0.9451 - val_accuracy: 0.9825 - val_loss: 0.0680
Epoch 2/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 2769s 3s/step - accuracy: 0.9937 - loss: 0.0272 - val_accuracy: 0.9931 - val_loss: 0.0251
Epoch 3/10
237/800 ━━━━━━━━━━━━━━━━━━━━ 31:06 3s/step - accuracy: 0.9929 - loss: 0.0266

Plot Training History

In [ ]:
plt.figure(figsize=(12, 5))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Accuracy vs. Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Loss vs. Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()

Evaluate the Model

In [ ]:
# Generate predictions for the test set
predictions = model.predict(test_data, verbose=1)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_data.classes

# Get class names from the generator (sorted by the generator's class_indices)
class_names = list(test_data.class_indices.keys())

# Print Classification Report
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_names))

# Compute Confusion Matrix and plot it
cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

Flask Web Application Integration

In [ ]:
from flask import Flask, request, jsonify
from PIL import Image
import io

app = Flask(__name__)
model = tf.keras.models.load_model(MODEL_SAVE_PATH)

# Define class labels as per your dataset (adjust the order if necessary)
class_labels = [
    "astroctioma", "carcinoma", "ependimoma", "ganglioglioma", "germinoma",
    "glioblastoma", "granuloma", "meduloblastoma", "meningioma", "neurocitoma",
    "no_tumor", "oligodendroglioma", "papiloma", "pituitary", "schwannoma", "tuberculoma"
]

def preprocess_image(image, target_size):
    # Convert image to RGB if necessary and resize
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target_size)
    image_array = np.array(image).astype("float32") / 255.0
    return np.expand_dims(image_array, axis=0)

@app.route("/", methods=["GET", "POST"])
def upload_predict():
    if request.method == "POST":
        if 'image' not in request.files:
            return jsonify({"error": "No image uploaded"}), 400
        image_file = request.files["image"].read()
        image = Image.open(io.BytesIO(image_file))
        processed_image = preprocess_image(image, target_size=(224, 224))
        preds = model.predict(processed_image)
        predicted_index = np.argmax(preds, axis=1)[0]
        predicted_label = class_labels[predicted_index]
        return jsonify({"predicted_label": predicted_label})

    return '''
    <html>
      <body>
        <h1>Upload an Image for Prediction</h1>
        <form method="post" enctype="multipart/form-data">
          <input type="file" name="image" accept="image/*">
          <input type="submit" value="Predict">
        </form>
      </body>
    </html>
    '''

if __name__ == "__main__":
    try:
        from flask_ngrok import run_with_ngrok
        run_with_ngrok(app)
    except ImportError:
        pass
    app.run(host='0.0.0.0', port=5000, debug=True)